<a href="https://colab.research.google.com/github/imirenechen/DS-Projects/blob/main/Project-6%20HOG_SVM_makeup_classifier_optimized_by_Optuna_heavy_makeup_CelebA_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please download the following zip file, extract it and put it under your google 
drive

https://tinyurl.com/bsff25hs


In this example, we will perform a binary classification for heavy makeup and no-heavy makeup images coming from CelebA.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# if you mount Google drive correctly, the following commands should be able to be executed correctly
#!ls /content/drive/
%cd "/content/drive/MyDrive/Colab Notebooks"
%cd "heavy_makeup_CelebA_s"
!ls

/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks/heavy_makeup_CelebA_s
train  val


In [ ]:
# step 1: preparing HOG feature vectors for heavy makeup & no-heavy makeup classification in training & validation, respectively
from skimage import data, color, feature
import skimage.data
import cv2
from google.colab.patches import cv2_imshow
import glob
import numpy as np
from timeit import default_timer as timer

# image size before cropping
resize_img_w = 256
resize_img_h = 256

# image size after cropping
crop_img_w = 224
crop_img_h = 224

# feature size for 224x224 image using default HOG settings from sklearn
HOG_dim = 54756

# the folder of training data in YOUR Google drive
training_heavy_makeup_imgs_dir = "/content/drive/MyDrive/Colab Notebooks/heavy_makeup_CelebA_s/train/heavy_makeup/"
training_non_heavy_makeup_imgs_dir = "/content/drive/MyDrive/Colab Notebooks/heavy_makeup_CelebA_s/train/no_heavy_makeup/"

# the folder of validation data in YOUR Google drive
validation_heavy_makeup_imgs_dir = "/content/drive/MyDrive/Colab Notebooks/heavy_makeup_CelebA_s/val/heavy_makeup/"
validation_non_heavy_makeup_imgs_dir = "/content/drive/MyDrive/Colab Notebooks/heavy_makeup_CelebA_s/val/no_heavy_makeup/"

# the file lists of training data
training_heavy_makeup_img_files = glob.glob(training_heavy_makeup_imgs_dir + '*.jpg')
training_no_heavy_makeup_img_files = glob.glob(training_non_heavy_makeup_imgs_dir + '*.jpg')

# the file lists of validation data
validation_heavy_makeup_img_files = glob.glob(validation_heavy_makeup_imgs_dir + '*.jpg')
validation_no_heavy_makeup_img_files = glob.glob(validation_non_heavy_makeup_imgs_dir + '*.jpg')

# the number of trainimg images: 1000 for postive and negative, respectively
training_heavy_makeup_img_file_num = len(training_heavy_makeup_img_files)
training_no_heavy_makeup_img_file_num = len(training_no_heavy_makeup_img_files)

# the number of validation images: 1000 for postive and negative, respectively
validation_heavy_makeup_img_file_num = len(validation_heavy_makeup_img_files)
validation_no_heavy_makeup_img_file_num = len(validation_no_heavy_makeup_img_files)

#print(len(training_heavy_makeup_img_files))
#print(training_no_heavy_makeup_img_files)
#print(validation_heavy_makeup_img_files)
#print(validation_no_heavy_makeup_img_files)

print("the number of heavy makeup images in training folder:{} ".format(training_heavy_makeup_img_file_num))
print("the number of no heavy makeup images in training folder:{} ".format(training_no_heavy_makeup_img_file_num))

print("the number of heavy makeup images in validation folder:{}".format(validation_heavy_makeup_img_file_num))
print("the number of no heavy makeup images in validation folder:{} ".format(validation_no_heavy_makeup_img_file_num))

def img_crop(img, resize_img_w, resize_img_h, target_img_w, target_img_h):
  #h, w, c =img.shape
  img = cv2.resize(img, (int(resize_img_w), int(resize_img_h)))  
  top_y = (resize_img_h-target_img_h)/2
  btm_y = resize_img_h-(resize_img_h-target_img_h)/2
  left_x = (resize_img_w-target_img_w)/2
  right_x = resize_img_w-(resize_img_w-target_img_w)/2
  cropped_img = img[int(top_y):int(btm_y),int(left_x):int(right_x) ]
  
  return cropped_img

HOG_postitive_matrix = np.zeros([int(training_heavy_makeup_img_file_num), HOG_dim])
HOG_negative_matrix = np.zeros([int(training_no_heavy_makeup_img_file_num), HOG_dim])

print("start to prepare HOG features for postive training images")
start = timer()
for i in range(0,int(training_heavy_makeup_img_file_num)):
  #print("i={}".format(i))
  img = cv2.imread(training_heavy_makeup_img_files[i])
  img_cropped = img_crop(img, resize_img_w, resize_img_h, crop_img_w, crop_img_h)
  image = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
  hog_vec = feature.hog(image)
  #print(hog_vec.shape)
  HOG_postitive_matrix[i,:] = hog_vec
end = timer()
print("finish preparing HOG features for postive training images and totally {}-seconds are consumed".format(end-start))


print("start to prepare HOG features for negative training images")
start = timer()
for i in range(0,int(training_no_heavy_makeup_img_file_num)):
  #print("i={}".format(i))
  img = cv2.imread(training_no_heavy_makeup_img_files[i])
  img_cropped = img_crop(img, resize_img_w, resize_img_h, crop_img_w, crop_img_h)
  image = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
  hog_vec = feature.hog(image)
  #print(hog_vec.shape)
  HOG_negative_matrix[i,:] = hog_vec
end = timer()
print("finish preparing HOG features for negative training images and totally {}-seconds are consumed".format(end-start))

# stack HOG postive & negative features vertically
training_matrix = np.vstack((HOG_postitive_matrix,HOG_negative_matrix))

# initialize the label matrix where top half is positive
label_matrix = np.zeros(int(training_heavy_makeup_img_file_num) + int(training_no_heavy_makeup_img_file_num))
label_matrix[0:int(training_heavy_makeup_img_file_num)] = 1

the number of heavy makeup images in training folder:200 
the number of no heavy makeup images in training folder:200 
the number of heavy makeup images in validation folder:40
the number of no heavy makeup images in validation folder:40 
start to prepare HOG features for postive training images
finish preparing HOG features for postive training images and totally 12.46017424400003-seconds are consumed
start to prepare HOG features for negative training images
finish preparing HOG features for negative training images and totally 47.05626225899999-seconds are consumed


In [ ]:
## Optuna is not installed in colab so we need to manually install it.
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.4 MB/s eta 0:00:00


In [ ]:
# training and testing function used by Optuna
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import optuna

def SVM_training_n_testing(trial):
    # hyperparameters to be optimized by Optuna in training SVM     
    cfg = {'degree' : trial.suggest_int('degree', 0, 10), 'C' : trial.suggest_uniform('C', 0.1, 100)}

    print("start to perform training")
    start = timer()

    ## poly
    model = SVC(kernel='poly', probability=True, degree=cfg['degree'], C=cfg['C'])
    model.fit(training_matrix, label_matrix)

    end = timer()
    print("training is done and totally {}-seconds are consumed".format(end-start))

    # step 3: accuracy estimation by classifying all the images in the validation folder

    # for positive validation data
    TP_num = 0
    for i in range(0,validation_heavy_makeup_img_file_num): 
      img = cv2.imread(validation_heavy_makeup_img_files[i])
      img_cropped = img_crop(img, resize_img_w, resize_img_h, crop_img_w, crop_img_h)
      image = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
      hog_vec = feature.hog(image)
      hog_vec = hog_vec.reshape(1,-1)
      labels = model.predict(hog_vec)
      #print(labels)

      if int(labels[0]) == 1:
          #print("this is TP")
          TP_num = TP_num + 1
    print("TP_num={}".format(TP_num))

    # for negative validation data
    TN_num = 0
    for i in range(0,validation_no_heavy_makeup_img_file_num): #
      img = cv2.imread(validation_no_heavy_makeup_img_files[i])
      img_cropped = img_crop(img, resize_img_w, resize_img_h, crop_img_w, crop_img_h)  
      image = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
      hog_vec = feature.hog(image)

      #print(hog_vec.shape)
      hog_vec = hog_vec.reshape(1,-1)
      # perform classification
      labels = model.predict(hog_vec)
      #print(labels)
      if int(labels[0]) == 0:
        #print("this is TP")
        TN_num = TN_num + 1

    print("TN_num={}".format(TN_num))
    accuracy = (TP_num+TN_num)/(validation_heavy_makeup_img_file_num + validation_no_heavy_makeup_img_file_num)
    print("accuracy={}".format(accuracy))

    return accuracy

In [ ]:
# step 2: performing (SVM) classifier training and testing using sklearn

sampler = optuna.samplers.TPESampler()      
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(func=SVM_training_n_testing, n_trials=10)

[I 2023-03-18 17:10:56,137] A new study created in memory with name: no-name-e48f5896-87c0-446f-b634-3ea2c723c438
<ipython-input-7-208d37ff6e8b>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  cfg = {'degree' : trial.suggest_int('degree', 0, 10), 'C' : trial.suggest_uniform('C', 0.1, 100)}


start to perform training
training is done and totally 40.90421566399999-seconds are consumed
TP_num=29


[I 2023-03-18 17:11:58,356] Trial 0 finished with value: 0.775 and parameters: {'degree': 10, 'C': 17.68905108531756}. Best is trial 0 with value: 0.775.


TN_num=33
accuracy=0.775
start to perform training


<ipython-input-7-208d37ff6e8b>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  cfg = {'degree' : trial.suggest_int('degree', 0, 10), 'C' : trial.suggest_uniform('C', 0.1, 100)}


...

TN_num=30
accuracy=0.8
start to perform training


<ipython-input-7-208d37ff6e8b>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  cfg = {'degree' : trial.suggest_int('degree', 0, 10), 'C' : trial.suggest_uniform('C', 0.1, 100)}


training is done and totally 28.92659469299997-seconds are consumed
TP_num=35


[I 2023-03-18 17:17:14,636] Trial 9 finished with value: 0.825 and parameters: {'degree': 1, 'C': 78.56738108008125}. Best is trial 0 with value: 0.775.


TN_num=31
accuracy=0.825


### 修改Optuna最佳化人臉分類的程式，使其最小化classfication accuracy
- 修改 optuna.create_study(sampler=sampler, **direction='minimize'**) 中的direction從'maximize'變成'minimize'
- Remark: 最小化classification accuracy並不代表不在意準確率，在實際應用中，可以採用其他的指標來評估，如precision、recall、F1-score等指標